In [1]:
#establish connection
import mysql.connector
conn = mysql.connector.connect(user='root', password='',
                           host='localhost', database='semtraclus')
print(conn)

In [2]:
#calculate stay points

from math import radians, cos, sin, asin, sqrt
import os
import glob
from datetime import datetime
import folium
from folium.plugins import MarkerCluster
stay_points = []
def read_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()[6:] # Skip the first 6 lines
        points = []
        for line in lines:
            lat, lon, _, _, _, _, timestamp = line.strip().split(',')
            timestamp = datetime.strptime(timestamp, '%H:%M:%S') # convert timestamp to datetime object
            points.append((float(lat), float(lon),timestamp))

        return points

def manhattan_distance(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def find_stay_points(file_path, threshold_distance, min_duration):
    points = read_file(file_path)
    stay_points = []
    current_stay_point = []
    for i in range(1, len(points)):
        dist = manhattan_distance(points[i][:2], points[i-1][:2])
        if dist <= threshold_distance:
            current_stay_point.append(points[i])
        else:
            if len(current_stay_point) > 0:
                # Check if the current stay point satisfies the minimum duration requirement
                duration = current_stay_point[-1][2] - current_stay_point[0][2]
                if duration.total_seconds() >= min_duration:
                    start_time = current_stay_point[0][2].strftime('%H:%M:%S')
                    end_time = current_stay_point[-1][2].strftime('%H:%M:%S')
                    stay_points.append((current_stay_point[0][:2], current_stay_point[-1][:2], duration.total_seconds(), start_time, end_time))
                current_stay_point = []
    return stay_points


if __name__ == '__main__':
    # Directory containing the .plt files
    directory = 'C:/Users/farha/single_traj/'
    # Threshold distance for defining a stay point
    threshold_distance = 0.005 # in degrees (approx. 555 meters)
    # Minimum duration for a stay point
    min_duration = 300 # in number of points
    
    all_stay_points = []
    # Initialize a folium Map object centered at the first point in the first file
    
    file_path = glob.glob(os.path.join(directory, '*.plt'))[0]
    points = read_file(file_path)
    m = folium.Map(location=[points[0][0], points[0][1]], zoom_start=12)
    
    
    # Loop through all .plt files in the directory and find stay points
    for file_path in glob.glob(os.path.join(directory, '*.plt')):
        stay_points = find_stay_points(file_path, threshold_distance, min_duration)
        for i, point in enumerate(stay_points):
            all_stay_points += (point,)
            #print(f'All stay points: {all_stay_points}')
       # print(f'Stay points in {file_path}:')
        for i, point in enumerate(stay_points):
            print(f'Stay points in {file_path}:')
            print(f'Stay point {i+1}: Latitude={point[0][0]}, Longitude={point[0][1]}, Duration={point[2]} seconds')
            folium.Marker(location=[point[0][0], point[0][1]], popup=f"lat: {point[0][0]}, long: {point[0][1]}").add_to(m)
    # Create a marker cluster
    marker_cluster = MarkerCluster().add_to(m)
    # Display the map
    display(m)


Stay points in C:/Users/farha/single_traj\20071208165625.plt:
Stay point 1: Latitude=39.9638133, Longitude=116.3160783, Duration=4328.0 seconds
Stay points in C:/Users/farha/single_traj\20071215093917.plt:
Stay point 1: Latitude=39.9025816, Longitude=116.4204216, Duration=1331.0 seconds
Stay points in C:/Users/farha/single_traj\20071215093917.plt:
Stay point 2: Latitude=39.90664, Longitude=116.3916233, Duration=2267.0 seconds
Stay points in C:/Users/farha/single_traj\20071215093917.plt:
Stay point 3: Latitude=39.9468633, Longitude=116.38745, Duration=9484.0 seconds
Stay points in C:/Users/farha/single_traj\20071215093917.plt:
Stay point 4: Latitude=39.93775, Longitude=116.3863483, Duration=5304.0 seconds
Stay points in C:/Users/farha/single_traj\20071215093917.plt:
Stay point 5: Latitude=39.9088166, Longitude=116.4050783, Duration=6554.0 seconds
Stay points in C:/Users/farha/single_traj\20071215093917.plt:
Stay point 6: Latitude=39.9027466, Longitude=116.4202183, Duration=821.0 seconds

Stay points in C:/Users/farha/single_traj\20080219123720.plt:
Stay point 1: Latitude=30.9408099, Longitude=117.8112983, Duration=617.0 seconds
Stay points in C:/Users/farha/single_traj\20080219123720.plt:
Stay point 2: Latitude=30.8622983, Longitude=117.6247933, Duration=802.0 seconds
Stay points in C:/Users/farha/single_traj\20080219123720.plt:
Stay point 3: Latitude=31.1140699, Longitude=117.2995566, Duration=1338.0 seconds
Stay points in C:/Users/farha/single_traj\20080219123720.plt:
Stay point 4: Latitude=31.25282, Longitude=117.2793233, Duration=993.0 seconds
Stay points in C:/Users/farha/single_traj\20080219123720.plt:
Stay point 5: Latitude=31.3432866, Longitude=117.1755433, Duration=6957.0 seconds
Stay points in C:/Users/farha/single_traj\20080219123720.plt:
Stay point 6: Latitude=31.3402783, Longitude=117.1742766, Duration=440.0 seconds
Stay points in C:/Users/farha/single_traj\20080219123720.plt:
Stay point 7: Latitude=31.3427366, Longitude=117.1721583, Duration=2421.0 second

In [4]:
#store stay points and find points revisited using 10 meter approx radius


cursor = conn.cursor()

# create a table for the stay points
staypoints_table_name = 'stay_points'
create_staypoints_table_query = f"""CREATE TABLE IF NOT EXISTS {staypoints_table_name} (
                        id INT AUTO_INCREMENT PRIMARY KEY,
                        location POINT NOT NULL,
                        duration INT,
                        loc_id VARCHAR(20)
                        );"""
cursor.execute(create_staypoints_table_query)

# create a table for the repeated rows
repeated_rows_table_name = 'repeated_rows'
create_repeated_rows_table_query = f"""CREATE TABLE IF NOT EXISTS {repeated_rows_table_name} (
                        id INT AUTO_INCREMENT PRIMARY KEY,
                        location POINT NOT NULL,
                        duration INT,
                        loc_id VARCHAR(20),
                        loc_ref VARCHAR(20)
                        );"""
cursor.execute(create_repeated_rows_table_query)

# insert the stay points into the staypoints table
insert_query = f"""INSERT INTO {staypoints_table_name} (location, duration, loc_id) 
                    VALUES (ST_PointFromText(%s), %s, %s)"""

for i, point in enumerate(all_stay_points):
    loc_id = f"{point[0][0]:.6f}{point[0][1]:.6f}"
    data = (f'POINT({point[0][1]} {point[0][0]})', point[2], loc_id)
    cursor.execute(insert_query, data)

# retrieve the stay points and print the results
cursor.execute("SELECT id,ST_AsText(location),duration,loc_id FROM stay_points")
staypoint_rows = cursor.fetchall()
for row in staypoint_rows:
    print(row)

# iterate through the staypoint_rows and check for points within 10 meters
for i, row1 in enumerate(staypoint_rows):
    for j, row2 in enumerate(staypoint_rows[i+1:]):
        # get the latitude and longitude coordinates of the two points
        lat1, lon1 = row1[1].replace("POINT(", "").replace(")", "").split(" ")
        lat2, lon2 = row2[1].replace("POINT(", "").replace(")", "").split(" ")
        #print(lat2,lon2)
        # calculate the distance between the two points in meters
        distance = 6371000 * 2 * asin(sqrt(sin(radians(float(lat2)-float(lat1))/2)**2 + cos(radians(float(lat1))) * cos(radians(float(lat2))) * sin(radians(float(lon2)-float(lon1))/2)**2))
        if distance < 17:  # if distance between two points is less than 17 meters
            # insert the repeated row into the repeated_rows table
            insert_repeated_row_query = f"""INSERT INTO {repeated_rows_table_name} (location, duration, loc_id,loc_ref) 
                                            VALUES (ST_PointFromText(%s), %s, %s,%s)"""
            cursor.execute(insert_repeated_row_query, (row2[1], row2[2], row2[3],row1[3]))

# retrieve and print the results from the repeated_rows table
cursor.execute(f"SELECT id,ST_AsText(location),duration,loc_id,loc_ref FROM {repeated_rows_table_name}")
repeated_rows = cursor.fetchall()
print("Repeated rows:")
for row in repeated_rows:
    print(row)

# commit the changes and close the connection
conn.commit()
conn.close()


(1, 'POINT(116.339 39.97715)', 1029, '39.977150116.339000')
(2, 'POINT(116.446 39.9809333333333)', 1426, '39.980933116.446000')
(3, 'POINT(116.463283333333 39.9905)', 465, '39.990500116.463283')
(4, 'POINT(116.4781 40.0088166666667)', 1674, '40.008817116.478100')
(5, 'POINT(116.338133333333 39.9763833333333)', 7220, '39.976383116.338133')
(6, 'POINT(116.444566666667 39.9075166666667)', 4677, '39.907517116.444567')
(7, 'POINT(116.347 39.9754)', 545, '39.975400116.347000')
(8, 'POINT(116.361916666667 39.9568666666667)', 5228, '39.956867116.361917')
(9, 'POINT(116.353816666667 39.9754666666667)', 9739, '39.975467116.353817')
(10, 'POINT(116.368966666667 39.9564666666667)', 2161, '39.956467116.368967')
(11, 'POINT(116.34385 39.9799)', 12345, '39.979900116.343850')
(12, 'POINT(116.361533333333 39.9566)', 18838, '39.956600116.361533')
(13, 'POINT(116.337116666667 39.9765333333333)', 22654, '39.976533116.337117')
(14, 'POINT(116.361683333333 39.9565666666667)', 2622, '39.956567116.361683')
(1

Repeated rows:
(1, 'POINT(116.361683333333 39.9565666666667)', 2622, '39.956567116.361683', '39.956600116.361533')
(2, 'POINT(116.337933333333 39.9768333333333)', 338, '39.976833116.337933', '39.976967116.338017')
(3, 'POINT(116.361683333333 39.9565666666667)', 2622, '39.956567116.361683', '39.956600116.361533')
(4, 'POINT(116.337933333333 39.9768333333333)', 338, '39.976833116.337933', '39.976967116.338017')
(5, 'POINT(116.2984933 39.9939966)', 773, '39.993997116.298493', '39.993713116.298512')
(6, 'POINT(116.361683333333 39.9565666666667)', 2622, '39.956567116.361683', '39.956600116.361533')
(7, 'POINT(116.337933333333 39.9768333333333)', 338, '39.976833116.337933', '39.976967116.338017')
(8, 'POINT(116.3160783 39.9638133)', 4328, '39.963813116.316078', '39.963813116.316078')
(9, 'POINT(116.4204216 39.9025816)', 1331, '39.902582116.420422', '39.902582116.420422')
(10, 'POINT(116.3916233 39.90664)', 2267, '39.906640116.391623', '39.906640116.391623')
(11, 'POINT(116.38745 39.9468633)'

In [5]:
#map the stay points and revisited points

import folium

# create a map centered on the first stay point
#map_center = staypoint_rows[0][1].replace("POINT(", "").replace(")", "").split(" ")
#m = folium.Map(location=[float(map_center[0]), float(map_center[1])], zoom_start=15)




# get the latitude and longitude of the first stay point
lat_lon = staypoint_rows[0][1].replace("POINT(", "").replace(")", "").split(" ")

# create a map centered on the first stay point

m = folium.Map(location=[float(lat_lon[1]),float(lat_lon[0])], zoom_start=15)


# add markers for the stay points
for row in staypoint_rows:
    lat_lon = row[1].replace("POINT(", "").replace(")", "").split(" ")
    folium.Marker(location=[float(lat_lon[1]), float(lat_lon[0])], popup=f"Duration: {row[2]}").add_to(m)

# add markers for the repeated rows with a different color
for row in repeated_rows:
    lat_lon = row[1].replace("POINT(", "").replace(")", "").split(" ")
    float_lat=float(lat_lon[0])
    float_lan=float(lat_lon[1])
    folium.Marker(location=[float(lat_lon[1]), float(lat_lon[0])], popup=f"Duration: {row[2]}, Loc Ref: {row[4]}", icon=folium.Icon(color='red')).add_to(m)
#folium.Marker(location=[point[0][0], point[0][1]], popup=f"lat: {point[0][0]}, long: {point[0][1]}").add_to(m)
    # Create a marker cluster
 
m

